In [22]:
import numpy as np
import matplotlib.pyplot as plt

In [23]:
#==PARAMETROS
EXP=1 #TODO: hacer promedios para poder correr durante algunos meses 
DIAS=20  #Una corrida equivale a un dia
incrementoStockM4= 50
incrementoStockM6= 30
#cantidad de minutos de 1 dia laboral de 8 horas
minutosXdia = 60*8
cantDiasProduccion = 10
#velocidad Operadores distribución Exponencial con media 5 min. para las partes de la mesa M4 
#y 10 min. para las partes de la mesa M6
# Si el valor es muy pequeño probar con 25 para M4 y 30 para M6
velOperM4=5
velOperM6=10
#==================

#== VARIABLES AUXILIARES
pedidos = []
stockM4=0
stockM6=0
pedidosAtendidos=0
pedidosNOatendidos=0
#stockM6=0
minutosRestantes=0 #Contador de minutos restantes en un dia 


solicitudesSinAtender=0
solicitudesAtendidas=0
mesasConstruidasM4=[]
mesasConstruidasM6=[]
tiemposOperarios=[]


In [24]:
class Pedido:
    def __init__(self, tipo):
        self.tipo = tipo
        
    def get_tipo(self):
        return self.tipo
    
    def __str__(self):
        return "Pedido - tipo:{}".format(self.tipo)

In [25]:
def calcular_porcentajes(solicitudesAtendidas, solicitudesSinAtender):
    total = solicitudesAtendidas + solicitudesSinAtender
    porc_atendidas = ( solicitudesAtendidas * 100 ) / total
    porc_sin_atender = ( solicitudesSinAtender * 100) / total
    return porc_atendidas, porc_sin_atender

In [26]:
def calcular_mejora(porc_1, porc_2):
    return abs(porc_1 - porc_2)

In [27]:
def simular(incrementoStockM4, incrementoStockM6, cantDiasProduccion):
    pedidos = []
    stockM4=0
    stockM6=0
    pedidosAtendidos=0
    pedidosNOatendidos=0
    #stockM6=0
    minutosRestantes=0 #Contador de minutos restantes en un dia 


    solicitudesSinAtender=0
    solicitudesAtendidas=0
    mesasConstruidasM4=[]
    mesasConstruidasM6=[]
    tiemposOperarios=[]

    for i in range(EXP):
        for dia in range(DIAS):
            minutosRestantes = minutosXdia
            mesasConstruidasM4.append(0)
            mesasConstruidasM6.append(0)
            tiemposOperarios.append(0)

            #Incremento el stock los promeros N dias
            if (dia <= cantDiasProduccion):
                stockM4 +=incrementoStockM4
                stockM6 +=incrementoStockM6

            cantPedidosM4 = np.random.poisson(8)
            cantPedidosM6 = np.random.poisson(4)

            #Armo una lista con elemntos M4 y M6
            for _ in range(cantPedidosM4):
                pedidos.append('M4')
            for _ in range(cantPedidosM6):    
                pedidos.append('M6')


            #Desordeno lista para no atender siempre las M4 primero
            np.random.shuffle(pedidos)    


            #Recorro la lista de solicitudes
            for item in pedidos:   
                if (minutosRestantes > 0):
                    if (item == 'M4'):                
                        if (stockM4 > 0):
                            #Decrementar stock M4
                            stockM4 -= 1

                            #Esperar tiempo de Operador. se resta al tiempo disponible                    
                            tiempoEspera = np.random.exponential(scale=velOperM4)
                            minutosRestantes -= tiempoEspera

                            #Contabilizar tiempo de Operarios (espera)
                            tiemposOperarios[dia] += tiempoEspera

                            #contabilizar solicitudes atendidas (por dia)
                            solicitudesAtendidas += 1

                            #Incrementamos contador de mesas contruidas M4                
                            mesasConstruidasM4[dia] += 1 

                            #Resto al tiempo restante la velocidad de extraccion M4
                            #minutosRestantes -= item.velocidad

                        else:                        
                            solicitudesSinAtender +=1

                    if (item == 'M6'):                
                        if (stockM6 > 0):
                            #Decrementar stock M4
                            stockM6 -= 1

                            #Esperar tiempo de Operador. se resta al tiempo disponible                    
                            tiempoEspera = np.random.exponential(scale=velOperM6)
                            minutosRestantes -= tiempoEspera

                            #Contabilizar tiempo de Operarios (espera)
                            tiemposOperarios[dia] += tiempoEspera

                            #contabilizar solicitudes atendidas (por dia)
                            solicitudesAtendidas += 1

                            #Incrementamos contador de mesas contruidas M6                
                            mesasConstruidasM6[dia] += 1 

                            #Resto al tiempo restante la velocidad de extraccion M6
                            #minutosRestantes -= item.velocidad

                        else:                        
                            solicitudesSinAtender +=1


                else: 
                    #Se terminaron los minutos del dia
                    #contabilizar solicitudes NO atendidas (por dia)
                    solicitudesSinAtender +=1
    return solicitudesAtendidas, solicitudesSinAtender

#### Simulamos con los parametros default

In [28]:
"""
print('Mesas M4 construidas por dia:')
for dia in range(DIAS):
    print('--DIA: %02d' % (dia + 1) + ' cantidad: ' + str(mesasConstruidasM4[dia] )) 

print('Mesas M6 construidas por dia:')
for dia in range(DIAS):
    print('--DIA: %02d' % (dia + 1) + ' cantidad: ' + str(mesasConstruidasM6[dia] )) 

print('Minutos totales de espera Operarios en Deposito por dia:')
for dia in range(DIAS):
    print('--DIA: %02d' % (dia + 1) + ' cantidad: %.0f' % (tiemposOperarios[dia]) ) 

"""

"\nprint('Mesas M4 construidas por dia:')\nfor dia in range(DIAS):\n    print('--DIA: %02d' % (dia + 1) + ' cantidad: ' + str(mesasConstruidasM4[dia] )) \n\nprint('Mesas M6 construidas por dia:')\nfor dia in range(DIAS):\n    print('--DIA: %02d' % (dia + 1) + ' cantidad: ' + str(mesasConstruidasM6[dia] )) \n\nprint('Minutos totales de espera Operarios en Deposito por dia:')\nfor dia in range(DIAS):\n    print('--DIA: %02d' % (dia + 1) + ' cantidad: %.0f' % (tiemposOperarios[dia]) ) \n\n"

In [29]:
solicitudesAtendidas, solicitudesSinAtender = simular(50,30,10)                  
                    
#RESULTADOS
porc_atendidas_5030, porc_sin_atender = calcular_porcentajes(solicitudesAtendidas, solicitudesSinAtender)

print('PARAMETRO Minutos totales de trabajo diario: ' + str(minutosXdia))
print('Solicitudes atendidas: {} ({:.2f}%) '.format(str(solicitudesAtendidas), porc_atendidas_5030))
print('Solicitudes NO atendidas: {} ({:.2f}%)'.format(str(solicitudesSinAtender), porc_sin_atender ))

PARAMETRO Minutos totales de trabajo diario: 480
Solicitudes atendidas: 880 (33.87%) 
Solicitudes NO atendidas: 1718 (66.13%)


In [30]:
solicitudesAtendidas, solicitudesSinAtender = simular(60,40,10)                  
                    
#RESULTADOS
porc_atendidas_6040, porc_sin_atender = calcular_porcentajes(solicitudesAtendidas, solicitudesSinAtender)

print('PARAMETRO Minutos totales de trabajo diario: ' + str(minutosXdia))
print('Solicitudes atendidas: {} ({:.2f}%) '.format(str(solicitudesAtendidas), porc_atendidas_6040))
print('Solicitudes NO atendidas: {} ({:.2f}%)'.format(str(solicitudesSinAtender), porc_sin_atender ))

PARAMETRO Minutos totales de trabajo diario: 480
Solicitudes atendidas: 1100 (43.75%) 
Solicitudes NO atendidas: 1414 (56.25%)


In [31]:
print("La mejora estimada es de un: {:.2f}%".format(calcular_mejora(porc_atendidas_5030, porc_atendidas_6040)))

La mejora estimada es de un: 9.88%
